# Import useful libs

In [ ]:
import numpy as np;
import pandas as pd;
import matplotlib as plt;
import geopandas as gpd

# Read Auxiliar CSVs

In [ ]:
anotate_codes = pd.read_csv('./datasets/anotate_codes.csv')
claim_codes = pd.read_csv('./datasets/claim_codes.csv')
commod_type_codes = pd.read_csv('./datasets/commod_type_codes.csv')
commodity_codes = pd.read_csv('./datasets/commodity_codes.csv')
concen_codes = pd.read_csv('./datasets/concen_codes.csv')
confmethod_codes = pd.read_csv('./datasets/confmethod_codes.csv')
country_codes = pd.read_csv('./datasets/country_codes.csv')
determin_codes = pd.read_csv('./datasets/determin_codes.csv')
diisttype_codes = pd.read_csv('./datasets/disttype_codes.csv')
extract_codes = pd.read_csv('./datasets/extract_codes.csv')
lab_codes = pd.read_csv('./datasets/lab_codes.csv')
mean_codes = pd.read_csv('./datasets/mean_codes.csv')
origin_codes = pd.read_csv('./datasets/origin_codes.csv')
pest_codes = pd.read_csv('./datasets/pest_codes.csv')
quantitate_codes = pd.read_csv('./datasets/quantitate_codes.csv')
state_codes = pd.read_csv('./datasets/state_codes.csv')
test_class_codes = pd.read_csv('./datasets/test_class_codes.csv')
tolerance_codes = pd.read_csv('./datasets/tolerance_codes.csv')
us_states = pd.read_csv('./datasets/us_states.csv')

# Read Samples from all years

In [365]:
samples_colums_labels = ('sample_pk','state', 'year', 'month', 'day', 'site', 'commod', 'source_id', 'variety', 'origin', 'country', 'disttype', 'commtype', 'claim', 'quantity', 'growst', 'packst', 'diistst');

samples_15 = pd.read_csv('./datasets/database/2015PDPDatabase/PDP15Samples.txt', delimiter="|", names=samples_colums_labels);
samples_16 = pd.read_csv('./datasets/database/2016PDPDatabase/PDP16Samples.txt', delimiter="|", names=samples_colums_labels);
samples_17 = pd.read_csv('./datasets/database/2017PDPDatabase/PDP17Samples.txt', delimiter="|", names=samples_colums_labels);
samples_18 = pd.read_csv('./datasets/database/2018PDPDatabase/PDP18Samples.txt', delimiter="|", names=samples_colums_labels);
samples_19 = pd.read_csv('./datasets/database/2019PDPDatabase/PDP19Samples.txt', delimiter="|", names=samples_colums_labels);

# Remove imported products
samples_15 = samples_15[samples_15['origin']==1]
samples_16 = samples_16[samples_16['origin']==1]
samples_17 = samples_17[samples_17['origin']==1]
samples_18 = samples_18[samples_18['origin']==1]
samples_19 = samples_19[samples_19['origin']==1]

# Remove unnecessary columns
samples_15.drop(columns=['country', 'growst', 'packst', 'diistst'], inplace = True) 
samples_16.drop(columns=['country', 'growst', 'packst', 'diistst'], inplace = True) 
samples_17.drop(columns=['country', 'growst', 'packst', 'diistst'], inplace = True) 
samples_18.drop(columns=['country', 'growst', 'packst', 'diistst'], inplace = True) 
samples_19.drop(columns=['country', 'growst', 'packst', 'diistst'], inplace = True) 

# Read Results from all years

In [ ]:
results_colums_labels = ('sample_pk','commod', 'commtype', 'lab', 'pestcode', 'testclass', 'concen', 'lod', 'conunit', 'confmethod', 'confmethod2', 'annotate', 'quantitate', 'mean', 'extract', 'determin');

In [ ]:
results_15 = pd.read_csv('./datasets/database/2015PDPDatabase/PDP15Results.txt', delimiter="|", names=results_colums_labels);
results_16 = pd.read_csv('./datasets/database/2016PDPDatabase/PDP16Results.txt', delimiter="|", names=results_colums_labels);
results_17 = pd.read_csv('./datasets/database/2017PDPDatabase/PDP17Results.txt', delimiter="|", names=results_colums_labels);
results_18 = pd.read_csv('./datasets/database/2018PDPDatabase/PDP18Results.txt', delimiter="|", names=results_colums_labels);
results_19 = pd.read_csv('./datasets/database/2019PDPDatabase/PDP19Results.txt', delimiter="|", names=results_colums_labels);

# Count each pesticide's presence in all years

In [ ]:
pest_count_general_15 = results_15["pestcode"].value_counts()
pest_count_general_16 = results_16["pestcode"].value_counts()
pest_count_general_17 = results_17["pestcode"].value_counts()
pest_count_general_18 = results_18["pestcode"].value_counts()
pest_count_general_19 = results_19["pestcode"].value_counts()

# Count each pesticide's presence per state in all years

In [ ]:
pest_state_join_15 = results_15.merge(samples_15,on='sample_pk',how='inner')[['pestcode', 'state']]
pest_state_count_15 = pest_state_join_15.value_counts().rename_axis(['pestcode', 'state']).reset_index(name='count')
pest_state_count_15 = pest_state_count_15.merge(us_states, on='state', how='inner')

In [ ]:
pest_state_join_16 = results_16.merge(samples_16,on='sample_pk',how='inner')[['pestcode', 'state']]
pest_state_count_16 = pest_state_join_16.value_counts().rename_axis(['pestcode', 'state']).reset_index(name='count')
pest_state_count_16 = pest_state_count_16.merge(us_states, on='state', how='inner')


In [ ]:
pest_state_join_17 = results_17.merge(samples_17,on='sample_pk',how='inner')[['pestcode', 'state']]
pest_state_count_17 = pest_state_join_17.value_counts().rename_axis(['pestcode', 'state']).reset_index(name='count')
pest_state_count_17 = pest_state_count_17.merge(us_states, on='state', how='inner')


In [ ]:
pest_state_join_18 = results_18.merge(samples_18,on='sample_pk',how='inner')[['pestcode', 'state']]
pest_state_count_18 = pest_state_join_18.value_counts().rename_axis(['pestcode', 'state']).reset_index(name='count')
pest_state_count_18 = pest_state_count_18.merge(us_states, on='state', how='inner')


In [ ]:
pest_state_join_19 = results_19.merge(samples_19,on='sample_pk',how='inner')[['pestcode', 'state']]
pest_state_count_19 = pest_state_join_19.value_counts().rename_axis(['pestcode', 'state']).reset_index(name='count')
pest_state_count_19 = pest_state_count_19.merge(us_states, on='state', how='inner')

# Count each pesticide's presence per commodity in all years

In [ ]:
pest_commod_count_15 = results_15[['pestcode','commod']].value_counts().rename_axis(['pestcode', 'commod']).reset_index(name='count')

In [ ]:
pest_commod_count_16 = results_16[['pestcode','commod']].value_counts().rename_axis(['pestcode', 'commod']).reset_index(name='count')

In [ ]:
pest_commod_count_17 = results_17[['pestcode','commod']].value_counts().rename_axis(['pestcode', 'commod']).reset_index(name='count')

In [ ]:
pest_commod_count_18 = results_18[['pestcode','commod']].value_counts().rename_axis(['pestcode', 'commod']).reset_index(name='count')

In [ ]:
pest_commod_count_19 = results_19[['pestcode','commod']].value_counts().rename_axis(['pestcode', 'commod']).reset_index(name='count')